
# Imports and load data

# Imports and load data

In [6]:
#imports
import pandas as pd
import numpy as np
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer



[nltk_data] Downloading package stopwords to /Users/tara-
[nltk_data]     sophiatumbraegel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/tara-
[nltk_data]     sophiatumbraegel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/tara-
[nltk_data]     sophiatumbraegel/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/tara-
[nltk_data]     sophiatumbraegel/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [7]:
# load data
df = pd.read_csv('../data/raw/mtsamples.csv')
df.transcription=df.transcription.astype(str)
#print(df.columns)
#df =df.head(15)

In [11]:
df.head()

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords
0,0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."
2,2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."


# Clean and Preprocess the data

In [19]:

# remove rows with missing values
def clean_df(data):
    df = data.dropna().drop_duplicates() 
    # drop rows where medical specialty appears less than 50 times
    df = df[df.groupby("medical_specialty")["medical_specialty"].transform('size') > 50]
    print("Number of rows after removing medical specialties with less than 50 samples:", len(df.index))
    # remove unnecessary columns, only keep transcriptions and medical_specialty columns
    return df[['transcription', 'medical_specialty']]


# remove punctuation and lowercase and lemmatizer
stop = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

def remove_punct_lower(data):
    '''remove punctuation and lowercase'''
    data["transcription_c"] = data["transcription"].apply(lambda x: x.lower().translate(str.maketrans('','', string.punctuation)))
    return data

def lemmatize_words(data):
    '''lemmatize words, remove stopwords'''
    data["transcription_c"] = data["transcription_c"].apply(lambda x: [lemmatizer.lemmatize(x) for x in word_tokenize(x) if x not in (stop)])
    return data
 


#apply on dataset
df_m = clean_df(df)
df_test = remove_punct_lower(df_m)
df_test = lemmatize_words(df_test)


df_test.head(2)


Number of rows after removing medical specialties with less than 50 samples: 3546


,transcription,medical_specialty,transcription_c
3,"2-D M-MODE: , ,1. Left atrial enlargement wit...",Cardiovascular / Pulmonary,"[2d, mmode, 1, left, atrial, enlargement, left..."
4,1. The left ventricular cavity size and wall ...,Cardiovascular / Pulmonary,"[1, left, ventricular, cavity, size, wall, thi..."


# NLP with Spacy
things to do:
- currently only single word extraction instead of multi keywords
- group by sickness
- pipeline
- valuation


In [ ]:
# install
#!python -m spacy download en_ner_bionlp13cg_md
# !pip3 install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_sm-0.5.1.tar.gz

#!pip3 install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_sm-0.5.1.tar.gz
#!pip3 install ../data/en_ner_bionlp13cg_md-0.5.1

In [23]:
df_small = df_test.head(10)
df_small

,transcription,medical_specialty,transcription_c
3,"2-D M-MODE: , ,1. Left atrial enlargement wit...",Cardiovascular / Pulmonary,"[2d, mmode, 1, left, atrial, enlargement, left..."
4,1. The left ventricular cavity size and wall ...,Cardiovascular / Pulmonary,"[1, left, ventricular, cavity, size, wall, thi..."
7,"2-D ECHOCARDIOGRAM,Multiple views of the heart...",Cardiovascular / Pulmonary,"[2d, echocardiogrammultiple, view, heart, grea..."
9,"DESCRIPTION:,1. Normal cardiac chambers size....",Cardiovascular / Pulmonary,"[description1, normal, cardiac, chamber, size2..."
11,"2-D STUDY,1. Mild aortic stenosis, widely calc...",Cardiovascular / Pulmonary,"[2d, study1, mild, aortic, stenosis, widely, c..."
16,"PREOPERATIVE DIAGNOSES,Airway obstruction seco...",Cardiovascular / Pulmonary,"[preoperative, diagnosesairway, obstruction, s..."
18,"PROCEDURE: , Elective male sterilization via b...",Urology,"[procedure, elective, male, sterilization, via..."
19,"HISTORY OF PRESENT ILLNESS:, The patient is a...",General Medicine,"[history, present, illness, patient, 17yearold..."
20,"INDICATION:, Prostate Cancer.,TECHNIQUE:, 3....",Urology,"[indication, prostate, cancertechnique, 35, ho..."
22,"DESCRIPTION:, The patient was placed in the s...",Urology,"[description, patient, placed, supine, positio..."


In [24]:
# NLP with Spacy
import spacy
import en_ner_bionlp13cg_md
nlp = en_ner_bionlp13cg_md.load()
def medical_entities( text):
    entities = []
    doc = nlp(text)
    for ent in doc.ents:
        entities.append(ent.text)
    return set(entities)#' ,'.join(entities)


df_small['transcription_c'] = [','.join(map(str, l)) for l in df_small['transcription_c']]
df_small['transcription_f'] = df_small['transcription_c'].apply(medical_entities)
df_small.head()

/var/folders/7p/lw9128713_9433llvvhnlv680000gn/T/ipykernel_3064/1039835220.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_small['transcription_c'] = [','.join(map(str, l)) for l in df_small['transcription_c']]
/var/folders/7p/lw9128713_9433llvvhnlv680000gn/T/ipykernel_3064/1039835220.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_small['transcription_f'] = df_small['transcription_c'].apply(medical_entities)


,transcription,medical_specialty,transcription_c,transcription_f
3,"2-D M-MODE: , ,1. Left atrial enlargement wit...",Cardiovascular / Pulmonary,"2d,mmode,1,left,atrial,enlargement,left,atrial...","{left, pulmonary, ventricular, valve, mitral}"
4,1. The left ventricular cavity size and wall ...,Cardiovascular / Pulmonary,"1,left,ventricular,cavity,size,wall,thickness,...","{root, left, ventricle, atrium, wall, pulmonar..."
7,"2-D ECHOCARDIOGRAM,Multiple views of the heart...",Cardiovascular / Pulmonary,"2d,echocardiogrammultiple,view,heart,great,ves...","{cardiac, left, atrium, venous, heart, pulmona..."
9,"DESCRIPTION:,1. Normal cardiac chambers size....",Cardiovascular / Pulmonary,"description1,normal,cardiac,chamber,size2,norm...","{cardiac, left, ventricular, valve, mitral}"
11,"2-D STUDY,1. Mild aortic stenosis, widely calc...",Cardiovascular / Pulmonary,"2d,study1,mild,aortic,stenosis,widely,calcifie...","{left, ventricle, heart, ventricular, mitral}"


# Pipeline

In [11]:
import spacy
import en_ner_bionlp13cg_md
# remove rows with missing values

def clean_df(data):
    df = data.dropna().drop_duplicates() 
    # drop rows where medical specialty appears less than 50 times
    df = df[df.groupby("medical_specialty")["medical_specialty"].transform('size') > 100]
    print("Number of rows after removing medical specialties with less than 50 samples:", len(df.index))
    # remove unnecessary columns, only keep transcriptions and medical_specialty columns
    return df[['transcription', 'medical_specialty']]


# remove punctuation and lowercase and lemmatizer

def remove_punct_lower(data):
    '''remove punctuation and lowercase'''
    data["transcription"] = data["transcription"].apply(lambda x: x.lower().translate(str.maketrans('','', string.punctuation)))
    return data

def lemmatize_words(data):
    '''lemmatize words, remove stopwords'''
    # remove punctuation and lowercase and lemmatizer
    stop = stopwords.words('english')
    lemmatizer = WordNetLemmatizer()
    data["transcription"] = data["transcription"].apply(lambda x: [lemmatizer.lemmatize(x) for x in word_tokenize(x) if x not in (stop)])
    return data
 
def list_to_string_f(data):
    data["transcription"] = [','.join(map(str, l)) for l in data['transcription']]
    return data

    #str1 = " " 
    #return (str1.join(s))
    
import spacy
import en_ner_bionlp13cg_md

def nlp_model(data):
    data["transcription_text"] = [','.join(map(str, l)) for l in data['transcription']]

# NLP with Spacy

    def medical_entities( text):
        nlp = en_ner_bionlp13cg_md.load()
        entities = []
        doc = nlp(text)
        for ent in doc.ents:
            entities.append(ent.text)
        return set(entities)
    data['transcription_final'] = data['transcription_text'].apply(medical_entities)
    return data
    
    #return set(entities)#' ,'.join(entities)


#df_small['transcription_c'] = [','.join(map(str, l)) for l in df_small['transcription_c']]
#df_small['transcription_f'] = df_small['transcription_c'].apply(medical_entities)
#df_small.head()
df_small = df.head(200)

In [4]:



clean_data = FunctionTransformer(clean_df, validate=False)
remove_punctation = FunctionTransformer(remove_punct_lower, validate=False)
lemmatize_thewords = FunctionTransformer(lemmatize_words, validate=False)
nlp_model_final_function = FunctionTransformer(nlp_model, validate=False)


nlp = en_ner_bionlp13cg_md.load()
pl = Pipeline(memory=None,
    steps=[
        ('cleandata', clean_data),
        ('removepunct', remove_punctation),
        ('lemmatize', lemmatize_thewords ),
      # ('list_to_stringf', list_to_string_f)#,
       # ('selector', get_numeric_data),
       ('modelf', nlp_model_final_function)
    ], verbose=False)


pl.fit(df_small)

Number of rows after removing medical specialties with less than 50 samples: 142


Pipeline(steps=[('cleandata',
                 FunctionTransformer(func=<function clean_df at 0x7fcaf7176b90>)),
                ('removepunct',
                 FunctionTransformer(func=<function remove_punct_lower at 0x7fcaf7176c20>)),
                ('lemmatize',
                 FunctionTransformer(func=<function lemmatize_words at 0x7fcaf7176cb0>)),
                ('modelf',
                 FunctionTransformer(func=<function nlp_model at 0x7fcaf7176dd0>))])

#NLP with YAKE
https://towardsdatascience.com/keyword-extraction-methods-the-overview-35557350f8bb
https://medium.com/@galeopsi/getting-started-nlp-9955b2cdba8c


In [5]:
x = pl.transform(df_small)
x

Number of rows after removing medical specialties with less than 50 samples: 142


# Yake


The duplication_threshold variable is limit the duplication of words in different keywords. You can set the deduplication_threshold value to 0.1 to avoid the repetition of words in keywords. If you set the deduplication_threshold value to 0.9, then repetition of words is allowed in keywords

In [14]:
import spacy
import en_ner_bionlp13cg_md
# remove rows with missing values
def clean_df(data):
    df = data.dropna().drop_duplicates() 
    # drop rows where medical specialty appears less than 50 times
    df = df[df.groupby("medical_specialty")["medical_specialty"].transform('size') > 50]
    print("Number of rows after removing medical specialties with less than 50 samples:", len(df.index))
    # remove unnecessary columns, only keep transcriptions and medical_specialty columns
    return df[['transcription', 'medical_specialty']]


# remove punctuation and lowercase and lemmatizer

def remove_punct_lower(data):
    '''remove punctuation and lowercase'''
    data["transcription"] = data["transcription"].apply(lambda x: x.lower().translate(str.maketrans('','', string.punctuation)))
    return data

def lemmatize_words(data):
    '''lemmatize words, remove stopwords'''
    # remove punctuation and lowercase and lemmatizer
    stop = stopwords.words('english')
    lemmatizer = WordNetLemmatizer()
    data["transcription"] = data["transcription"].apply(lambda x: [lemmatizer.lemmatize(x) for x in word_tokenize(x) if x not in (stop)])
    return data
 
def list_to_string_f(data):
    data["transcription"] = [' '.join(map(str, l)) for l in data['transcription']]
    return data

# NLP with Spacy

def medical_entities( text):
    nlp = en_ner_bionlp13cg_md.load()
    entities = []
    doc = nlp(text)
    for ent in doc.ents:
        entities.append(ent.text)
    return set(entities)

def nlp_model(data):
    data['transcription_final'] = data['transcription'].apply(medical_entities)
    return data
    
    #return set(entities)#' ,'.join(entities)

df_small = df.head(200)

clean_data = FunctionTransformer(clean_df, validate=False)
remove_punctation = FunctionTransformer(remove_punct_lower, validate=False)
lemmatize_thewords = FunctionTransformer(lemmatize_words, validate=False)
list_to_string_words = FunctionTransformer(list_to_string_f, validate=False)
nlp_model_final_function = FunctionTransformer(nlp_model, validate=False)


pl = Pipeline(memory=None,
    steps=[
        ('cleandata', clean_data),
        ('removepunct', remove_punctation),
        ('lemmatize', lemmatize_thewords ),
       ('list_to_stringf', list_to_string_words)#,
      #  ('modelf', nlp_model_final_function)
    ], verbose=False)


x = pl.fit_transform(df_small)
x.head()

Number of rows after removing medical specialties with less than 50 samples: 142


,transcription,medical_specialty
18,procedure elective male sterilization via bila...,Urology
20,indication prostate cancertechnique 35 hour fo...,Urology
22,description patient placed supine position pre...,Urology
23,preoperative diagnosis voluntary sterilitypost...,Urology
25,diagnosis desire vasectomyname operation vasec...,Urology


In [15]:
import yake
def yake_function(text):
    entities = []
    kw_extractor = yake.KeywordExtractor()
    language = "en"
    max_ngram_size = 3
    deduplication_threshold = 0.9
    numOfKeywords = 20
    custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)
    keywords = custom_kw_extractor.extract_keywords(text)
    for kw in keywords:
        entities.append(kw[0])
        #print(kw)    
    return set(entities)

def yake_model(data):
    data['transcription_final'] = data['transcription'].apply(yake_function)
    return data

#y = FunctionTransformer(yake_model, validate=False)
y = yake_model(x)
y.head()

,transcription,medical_specialty,transcription_final
18,procedure elective male sterilization via bila...,Urology,"{hemostasis cautery end, double back securing,..."
20,indication prostate cancertechnique 35 hour fo...,Urology,"{indication prostate cancertechnique, skull up..."
22,description patient placed supine position pre...,Urology,"{sheath incised scalpel, position prepped drap..."
23,preoperative diagnosis voluntary sterilitypost...,Urology,"{clipped proximally distally, supine position ..."
25,diagnosis desire vasectomyname operation vasec...,Urology,"{chromic bleeding identifiedthrough, tied drop..."


In [16]:
from rake_nltk import Rake
def rake_nltk_function(text):
    rake_nltk_var = Rake()
    entities = []
    rake_nltk_var.extract_keywords_from_text(text)
    keywords_scored = rake_nltk_var.get_word_degrees()
    #keyword_extracted = rake_nltk_var.get_ranked_phrases()
    for key in keywords_scored:
        entities.append(key)
    return set(entities)

def rake_model(data):
    data['transcription_final'] = data['transcription'].apply(rake_nltk_function)
    return data

t = rake_model(x)
t.head()

,transcription,medical_specialty,transcription_final
18,procedure elective male sterilization via bila...,Urology,"{carefully, discussed, 1inch, tied, via, level..."
20,indication prostate cancertechnique 35 hour fo...,Urology,"{activity, clinically, administration, mci, re..."
22,description patient placed supine position pre...,Urology,"{catgut, incision, va, procedure, clamp, hemoc..."
23,preoperative diagnosis voluntary sterilitypost...,Urology,"{room, discussed, clamp, chromic, hemiscrotum,..."
25,diagnosis desire vasectomyname operation vasec...,Urology,"{incision, va, surrounding, room, procedure, t..."


In [ ]:
pd.set_option('max_colwidth', None)  # Remove any limitation on length 
                                     # of text displayed in a cell
pd.set_option('max_rows', 300)  # Display up to 300 rows in a dataset
